In [53]:
import csv
import subprocess
import getpass
import os
import gzip
from os import listdir
from os.path import isfile, join
from SciServer import Authentication

myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = getpass.getpass('Password for ' + myUserName + ': ')
userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = input("Please enter the name of your SAFE Desktop project folder : ") 
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname
devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

Password for mkim239:  ········
Please enter the name of your SAFE Desktop project folder :  LCICM


mkdir: cannot create directory '/home/idies/workspace/SAFE': File exists


In [54]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [55]:
nk_features = pd.read_csv("SAFE/neurokit2_ecg_features.csv")
nk_features = nk_features.iloc[1:, :]

In [56]:
hrv_features = pd.read_csv("SAFE/adv_features_labeled.csv")
combined_features = pd.merge(nk_features, hrv_features)
combined_features_cleaned = combined_features.drop(["hf", "hf_type", "subject_id", "QRS_Amp_II"], axis = 1)
df_cleaned = add_constant(combined_features_cleaned)

In [57]:
# impute nan/inf values
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in df_cleaned.columns:
    df_cleaned.fillna({col: df_cleaned[col].mean()}, inplace=True)

vif_data = pd.DataFrame()
vif_data["feature"] = df_cleaned.columns
vif_data["VIF"] = [variance_inflation_factor(df_cleaned.values, i) for i in range(df_cleaned.shape[1])]

/home/idies/miniconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/home/idies/miniconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [58]:
vif_data = vif_data.iloc[1:, :]  

In [59]:
for el in vif_data["VIF"]:
    print(el)    

3.2284887731016902
5.21407631722221
inf
inf
inf
inf
inf
inf
inf
inf
inf
23.727769998425874
nan
7.07479063226482
662.2952562332139
5867.5984120630865
22203.78307387663
3813.138644472121
5285.707705971304
1199.1890940844025
1248.371318085469
15913.723885425648
2609.3719732393215
4426.704035650139
1082.6667630665777
86.23739980100505
144.24158030950014
14202.507706252605
4393.741133603206
714.039419985342
980.0627936411105
15641.31799516487
1131.787421359314
3619.6086741305753
1095.1559283561985
1588.555114860779
8826.80668826849
15921.334453048976
3396.2845418288643
555.9465209693806
1754.4444892663732
15210.441234708323
6674.590118821344
3442.3973363336295
1.8430435605772304
3.4790525533202215
4.164481096847604
3.8927157845538733
3.6596582455824755
1.697041866342267
3.654072830009215
3.6199160830100086
4.524439008229688
3.970790494501613
2.4822174131319588
7.389585022009931
4.370403370226008
10.138350797944325
5.9537897137975895
3.181539464992154
8.518943394313716
7.3249761195055605
13.

In [60]:
# calculate # of VIF above 10
# calcualte # of VIF above 5
# rest are "regular features"

# try again but remove the INF features and recalculate

# summarizse in slides: what is VIF, common cutoffs, how to move forward
vif_10 = sum(vif_data["VIF"] > 10)
print("Between 10 and inf: " + str(vif_10))
vif_5 = sum((vif_data["VIF"] > 5) & (vif_data["VIF"] < 10))
print("Between 5 and 10: " + str(vif_5))
print("Between 0 and 5: " + str(len(vif_data["VIF"]) - vif_10 - vif_5))

print(vif_data[(vif_data["VIF"] > 5) & (vif_data["VIF"] < 10)])
print(vif_data[(vif_data["VIF"] >= 0) & (vif_data["VIF"] < 5)])

Between 10 and inf: 342
Between 5 and 10: 20
Between 0 and 5: 24
                feature       VIF
2                    HR  5.214076
14           Tpeak_Tend  7.074791
56             Q_Amp_V3  7.389585
59             T_Amp_V3  5.953790
61             Q_Amp_V4  8.518943
62             R_Amp_V4  7.324976
64             T_Amp_V4  7.965020
66             Q_Amp_V5  6.265617
69             T_Amp_V5  8.856001
72             R_Amp_V6  7.703512
74             T_Amp_V6  5.876109
99       sd_ratio_lead1  6.969957
148     tri_index_lead3  6.374050
174     tri_index_lead4  6.174981
200     tri_index_lead5  7.294173
226     tri_index_lead6  8.779015
252     tri_index_lead7  6.381264
256  ellipse_area_lead7  8.926027
307      sd_ratio_lead9  6.354684
333     sd_ratio_lead10  8.345030
            feature       VIF
1            RR_Int  3.228489
45         P_Amp_V1  1.843044
46         Q_Amp_V1  3.479053
47         R_Amp_V1  4.164481
48         S_Amp_V1  3.892716
49         T_Amp_V1  3.659658
50         

In [65]:
feature_good_vif = vif_data[vif_data["VIF"] <= 10]
feature_list = feature_good_vif["feature"].tolist()
print(feature_list)

['RR_Int', 'HR', 'Tpeak_Tend', 'P_Amp_V1', 'Q_Amp_V1', 'R_Amp_V1', 'S_Amp_V1', 'T_Amp_V1', 'P_Amp_V2', 'Q_Amp_V2', 'R_Amp_V2', 'S_Amp_V2', 'T_Amp_V2', 'P_Amp_V3', 'Q_Amp_V3', 'R_Amp_V3', 'T_Amp_V3', 'P_Amp_V4', 'Q_Amp_V4', 'R_Amp_V4', 'T_Amp_V4', 'P_Amp_V5', 'Q_Amp_V5', 'T_Amp_V5', 'P_Amp_V6', 'Q_Amp_V6', 'R_Amp_V6', 'S_Amp_V6', 'T_Amp_V6', 'sd_ratio_lead1', 'tri_index_lead3', 'sd_ratio_lead3', 'tri_index_lead4', 'sd_ratio_lead4', 'tri_index_lead5', 'sd_ratio_lead5', 'tri_index_lead6', 'sd_ratio_lead6', 'tri_index_lead7', 'sd_ratio_lead7', 'ellipse_area_lead7', 'sd_ratio_lead9', 'sd_ratio_lead10']


In [69]:
df = pd.read_csv("SAFE/complete_features.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
feature_cols = df.columns[~df.columns.str.contains("copy") & (df.columns != "hf_original")]
X_full = df[feature_cols]
X = X_full[feature_list]
y = df["hf_original"]

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

In [82]:
# get training accuracy
y_pred_train = model.predict(X_train_scaled)
accuracy_train = accuracy_score(y_train, y_pred_train)

In [83]:
print("Accuracy for test logistic regression): " + str(accuracy))
print("Accuracy for train logistic regression: " + str(accuracy_train))

Accuracy for test logistic regression): 0.7514001244555072
Accuracy for train logistic regression: 0.7587356628434249


In [84]:
# doing hyperparameter tuning
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2", "elasticnet", "None"],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}

In [76]:
# keeps throwing warnings otherwise
!pip3 install --upgrade pandas

In [85]:
import warnings
warnings.filterwarnings("ignore")
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


/home/idies/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/idies/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/idies/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/idies/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/idies/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires ver

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'None'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy', verbose=1)

In [80]:
print("Best parameters for logistic regression: ", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_scaled)
tuned_accuracy = accuracy_score(y_test, y_pred_tuned)
print("Accuracy of hyperparameter runed logistic regression: " + str(tuned_accuracy))

Best parameters for logistic regression:  {'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}
Accuracy of hyperparameter runed logistic regression: 0.7514001244555072
